In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv("../Data/flight_dep_tree.csv")
df.shape
df.head(5)

,MONTH,DAY_OF_MONTH_sin,DAY_OF_MONTH_cos,DAY_OF_WEEK_sin,DAY_OF_WEEK_cos,CRS_ELAPSED_TIME,DISTANCE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyVisibility,HourlyWindSpeed,CRS_DEP_TIME_sin,CRS_DEP_TIME_cos,OP_UNIQUE_CARRIER,ORIGIN,DEST,DEP_DELAY
0,4,0.201299,0.97953,0.974928,-0.222521,168.0,1020.0,33.0,36.0,89.0,5.0,10.0,0.990389,-0.138309,0,1,62,-5.0
1,4,0.201299,0.97953,0.974928,-0.222521,160.0,1020.0,33.0,45.0,63.0,10.0,10.0,-0.785317,-0.619094,0,1,62,15.0
2,4,0.201299,0.97953,0.974928,-0.222521,299.0,2279.0,37.0,43.0,80.0,10.0,6.0,0.939991,-0.341200,0,4,14,-3.0
3,4,0.201299,0.97953,0.974928,-0.222521,300.0,2279.0,41.0,49.0,74.0,10.0,6.0,-0.460974,-0.887413,0,4,14,-3.0
4,4,0.201299,0.97953,0.974928,-0.222521,300.0,2279.0,41.0,45.0,86.0,10.0,7.0,-0.500000,0.866025,0,4,14,-6.0


In [3]:
X = df.drop(columns=["DEP_DELAY"])
y = df["DEP_DELAY"].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

Train size: (36124, 17), Test size: (9031, 17)


In [5]:
param_grid = {
    'n_estimators': [100, 200, 300],       
    'max_depth': [10, 20, None],           
    'min_samples_split': [2, 5, 10],       
    'min_samples_leaf': [1, 2, 4]          
}

rf = RandomForestRegressor(random_state=42, n_jobs=-1)

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,                 
    scoring='r2',          
    n_jobs=-1,            
    verbose=2
)


In [6]:
print("\n Searching for the best hyperparameters using cross-validation (cv=3)")
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
print(f"Best CV R² Score: {grid.best_score_:.4f}")


 Searching for the best hyperparameters using cross-validation (cv=3)
Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
Best CV R² Score: 0.1651


In [8]:
best_rf = grid.best_estimator_
y_pred = best_rf.predict(X_test)

mae  = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("\n Random Forest Performance:")
print(f"MAE   = {mae:.3f}")
print(f"RMSE  = {rmse:.3f}")

print("Predictions:", y_pred[:11])
print("Actuals:    ", y_test[:11])


 Random Forest Performance:
MAE   = 7.001
RMSE  = 9.506
Predictions: [-4.07596013 -3.14177122 -6.60312496  6.53092944  5.9430447   4.2520012
 -2.45984051 -2.29848465 -1.11249032 -0.17618992  3.58036285]
Actuals:     [  3.  -8.  -9.   5.   7.  -4. -10.  -6.   0.  -2.  29.]
